# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
databases = mongo.list_database_names()

databases

['admin', 'config', 'local', 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
collections = db.list_collection_names()

collections

['establishments']

In [7]:
# review a document in the establishments collection

establishment_collection = db['establishments']

document = establishment_collection.find_one()

document

{'_id': ObjectId('662ea8edd0166b71e2d08e29'),
 'FHRSID': 1034540,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode'

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    'BusinessName': 'Penang Flavours',
}

In [10]:
# Insert the new restaurant into the collection
add_one = establishments.insert_one(new_restaurant)

In [12]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}

find_restaurant = establishments.find_one(query)

find_restaurant

{'_id': ObjectId('662ead3812893d0921a6a40a'),
 'BusinessName': 'Penang Flavours'}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}

projection = {'BusinessTypeID': 1, 'BusinessType': 1, '_id': 0}

business_type = establishments.find_one(query, projection)

business_type

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID
query = {'BusinessName': 'Penang Flavours'}

new_data = {
    '$set': {
        'BusinessTypeID': '1'
    }
}

result = establishments.update_one(query, new_data)

In [16]:
# Confirm that the new restaurant was updated
update = establishments.find_one(query)

update

{'_id': ObjectId('662ead3812893d0921a6a40a'),
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': '1'}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [18]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}

count = establishments.count_documents(query)

count

994

In [19]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_dover = establishments.delete_many(query)

In [21]:
# Check if any remaining documents include Dover
count = establishments.count_documents(query)

count

0

In [23]:
# Check that other documents remain with 'find_one'
others = establishments.find_one()

others

{'_id': ObjectId('662ea8edd0166b71e2d0910c'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [31]:
# Change the data type from String to Decimal for longitude and latitude
query = {}

documents = establishments.find(query)

for document in documents:
    
    if 'geocode' in document and 'longitude' in document['geocode'] and 'latitude' in document['geocode']:
        longitude = float(document['geocode']['longitude'])
        latitude = float(document['geocode']['latitude'])
    
        update_query = {
            '$set': {
                'geocode.longitude': longitude,
                'geocode.latitude': latitude
            }
        }
    
        establishments.update_many({'_id': document['_id']}, update_query)

Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [35]:
# Change the data type from String to Integer for RatingValue
query = {}

update = [
    {
        "$set": {
            "RatingValue": {
                "$cond": {
                    "if": {"$eq": ["$RatingValue", None]},
                    "then": None,
                    "else": {"$toInt": "$RatingValue"}
                }
            }
        }
    }
]

result = establishments.update_many(query, update)

In [36]:
# Check that the coordinates and rating value are now numbers
sample = establishments.find_one()

print("Data type of longitude:", type(sample['geocode']['longitude']))
print("Data type of latitude:", type(sample['geocode']['latitude']))
print("Data type of RatingValue:", type(sample['RatingValue']))

Data type of longitude: <class 'float'>
Data type of latitude: <class 'float'>
Data type of RatingValue: <class 'NoneType'>
